In [23]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy 
import random
import requests
from config import weather_api_key
from datetime import datetime
import json
import time


In [24]:
#Create a set of random latitude and longitude combinations
lats=np.random.uniform(low=-90, high=90, size=1500)
lngs=np.random.uniform(low=-180, high=180, size=1500)
lat_lngs=zip(lats,lngs)
lat_lngs

In [25]:
# add the latitudes and longitudes to a list
coordinates=list(lat_lngs)

In [27]:
#create a list for holding the cities
cities=[]
#identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    #if the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count to confirm sufficient count
len(cities)

615

In [28]:
# Starting URL for Weather Map API Call.
    # Per city - current weather
# https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={your api key}
url= "https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key

    # Per city - 3h historical data
# Set current time for historical data
today=int(time.time())
today
# https://api.openweathermap.org/data/2.5/onecall/timemachine?units=imperial&lat={lat}&lon={lon}&dt={time}&appid={YOUR API KEY}
rs_url = "https://api.openweathermap.org/data/2.5/onecall/timemachine?units=Imperial&appid=" + weather_api_key + '&dt=' + str(today)

In [76]:
city_data=[]

#print the begining of the logging
print('Beginning Data Retrieval    ')
print('----------------------------')

#create counters
record_count=1
set_count=1

#loop through all the cities in our list
for i, city in enumerate(cities):
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i>=50):
        set_count+=1
        record_count=1
    #Create endpoint URL with each city
    city_url=url+'&q='+city.replace(" ","+")
    #Log tje URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #Add 1 to the record count
    record_count+=1
    
#API request for each city - current weather
    try:
        city_weather=requests.get(city_url).json()
        city_lat=city_weather['coord']['lat']
        city_lng=city_weather['coord']['lon']
        city_max_temp=city_weather['main']['temp_max']
        city_humidity=city_weather['main']['humidity']
        city_clouds=city_weather['clouds']['all']
        city_wind=city_weather['wind']['speed']
        city_country=city_weather['sys']['country']
        city_description=city_weather['weather'][0]['description']

#API request for each city - historical data 
        city_rs_url= rs_url+ '&lat=' + str(city_lat) + '&lon=' + str(city_lng)
        city_rs_weather=requests.get(city_rs_url).json()

        #get data for the last 3 hours
        for num in range(0,3):
            #Rain data
            try:
                rain_1h=city_rs_weather['hourly'][0]['rain']['1h'] #Rain for T-1
            except:
                rain_1h=0 #If its not raining then set 0
            try:
                rain_2h=city_rs_weather['hourly'][1]['rain']['1h'] #Rain for T-2
            except:
                rain_2h=0 #If its not raining then set 0
            try:
                rain_3h=city_rs_weather['hourly'][2]['rain']['1h'] #Rain for T-3
            except:
                rain_3h=0 #If its not raining then set 0
                
            #The rain default is mm -> get rain in inches
            try:
                rain_inches=(rain_1h+rain_2h+rain_3h)/25.4
            except:
                rain_inches=0  #If its not raining then set 0

            #Snow data 
            try:
                snow_1h=city_rs_weather['hourly'][0]['snow']['1h']
            except:
                snow_1h=0
            try:
                snow_2h=city_rs_weather['hourly'][1]['snow']['1h']
            except:
                snow_2h=0
            try:
                snow_3h=city_rs_weather['hourly'][2]['snow']['1h']
            except:
                snow_3h=0

            try:
                snow_inches=(snow_1h+snow_2h+snow_3h)/25.4
            except:
                snow_inches=0


        #append the city information
        city_data.append({'City':city.title(),
                          'Country':city_country,
                         'Lat':city_lat,
                         'Lng':city_lng,
                         'Max Temp':city_max_temp,
                         'Humidity':city_humidity,
                         'Cloudiness':city_clouds,
                         'Wind Speed':city_wind,
                         "Current Description":city_description,
                          'Rain (inches)':rain_inches,
                         'Snow (inches)': snow_inches
                         })

#If an error is experienced, skip the city
    except:
        print(f"City not found. Skipping...")
        pass


city_data_df=pd.DataFrame(city_data)
city_data_df

Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | hilo
Processing Record 2 of Set 1 | te anau
Processing Record 3 of Set 1 | agadez
Processing Record 4 of Set 1 | byron bay
Processing Record 5 of Set 1 | hobart
Processing Record 6 of Set 1 | lakatoro
Processing Record 7 of Set 1 | wasilla
Processing Record 8 of Set 1 | puerto maldonado
Processing Record 9 of Set 1 | punta arenas
Processing Record 10 of Set 1 | ribeira grande
Processing Record 11 of Set 1 | amderma
City not found. Skipping...
Processing Record 12 of Set 1 | atuona
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | fortuna
Processing Record 15 of Set 1 | dujuma
City not found. Skipping...
Processing Record 16 of Set 1 | tiksi
Processing Record 17 of Set 1 | puerto ayora
Processing Record 18 of Set 1 | orlik
Processing Record 19 of Set 1 | ketchikan
Processing Record 20 of Set 1 | alta floresta
Processing Record 21 of Set 1 | kendari
Processing Record 22 of Set

City not found. Skipping...
Processing Record 38 of Set 4 | avarua
Processing Record 39 of Set 4 | gijon
Processing Record 40 of Set 4 | isangel
Processing Record 41 of Set 4 | bluff
Processing Record 42 of Set 4 | oranjemund
Processing Record 43 of Set 4 | miranorte
City not found. Skipping...
Processing Record 44 of Set 4 | lagoa
Processing Record 45 of Set 4 | slavsk
Processing Record 46 of Set 4 | padang
Processing Record 47 of Set 4 | zyryanka
Processing Record 48 of Set 4 | bargal
City not found. Skipping...
Processing Record 49 of Set 4 | linjiang
Processing Record 50 of Set 4 | castelo do piaui
Processing Record 1 of Set 5 | merced
Processing Record 2 of Set 5 | serdobsk
Processing Record 3 of Set 5 | mahenge
Processing Record 4 of Set 5 | viligili
City not found. Skipping...
Processing Record 5 of Set 5 | tornio
Processing Record 6 of Set 5 | itarema
Processing Record 7 of Set 5 | apiai
Processing Record 8 of Set 5 | karimganj
Processing Record 9 of Set 5 | castro
Processing R

Processing Record 25 of Set 8 | victoria
Processing Record 26 of Set 8 | pitimbu
Processing Record 27 of Set 8 | trincomalee
Processing Record 28 of Set 8 | phalodi
Processing Record 29 of Set 8 | yining
Processing Record 30 of Set 8 | luanda
Processing Record 31 of Set 8 | prieska
Processing Record 32 of Set 8 | meulaboh
Processing Record 33 of Set 8 | taga
Processing Record 34 of Set 8 | sisimiut
Processing Record 35 of Set 8 | taoudenni
Processing Record 36 of Set 8 | horsham
Processing Record 37 of Set 8 | turukhansk
Processing Record 38 of Set 8 | bataraza
City not found. Skipping...
Processing Record 39 of Set 8 | honiara
Processing Record 40 of Set 8 | grindavik
Processing Record 41 of Set 8 | ylivieska
Processing Record 42 of Set 8 | evensk
Processing Record 43 of Set 8 | goure
Processing Record 44 of Set 8 | lima duarte
Processing Record 45 of Set 8 | sayyan
Processing Record 46 of Set 8 | mujiayingzi
Processing Record 47 of Set 8 | svetlogorsk
Processing Record 48 of Set 8 | 

Processing Record 15 of Set 12 | nantucket
Processing Record 16 of Set 12 | pimentel
Processing Record 17 of Set 12 | hermiston
Processing Record 18 of Set 12 | ngukurr
City not found. Skipping...
Processing Record 19 of Set 12 | tarakan
Processing Record 20 of Set 12 | boffa
Processing Record 21 of Set 12 | batagay-alyta
Processing Record 22 of Set 12 | ovsyanka
Processing Record 23 of Set 12 | homer
Processing Record 24 of Set 12 | bolshiye uki
City not found. Skipping...
Processing Record 25 of Set 12 | umm ruwabah
City not found. Skipping...
Processing Record 26 of Set 12 | tessalit
Processing Record 27 of Set 12 | avera
Processing Record 28 of Set 12 | tottori
Processing Record 29 of Set 12 | bambous virieux
Processing Record 30 of Set 12 | kachug
Processing Record 31 of Set 12 | eureka
Processing Record 32 of Set 12 | sao felix do xingu
Processing Record 33 of Set 12 | dawei
Processing Record 34 of Set 12 | kaniama
Processing Record 35 of Set 12 | ayan
Processing Record 36 of Set

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.73,-155.09,78.80,61,1,4.70,clear sky,0.000000,0.0
1,Te Anau,NZ,-45.42,167.72,46.06,79,3,3.09,clear sky,0.000000,0.0
2,Agadez,NE,19.75,10.25,88.11,16,5,7.25,clear sky,0.000000,0.0
3,Byron Bay,AU,-28.65,153.62,75.20,53,29,10.29,scattered clouds,0.000000,0.0
4,Hobart,AU,-42.88,147.33,59.00,54,75,5.82,broken clouds,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
561,Boke,GN,10.94,-14.30,72.82,95,100,2.30,moderate rain,1.077953,0.0
562,Metro,ID,-5.11,105.31,88.16,64,89,4.45,light rain,0.005906,0.0
563,Praia Da Vitoria,PT,38.73,-27.07,62.60,72,40,17.18,scattered clouds,0.000000,0.0
564,Lazarev,RU,52.23,141.51,39.78,96,96,13.53,overcast clouds,0.000000,0.0


In [79]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [80]:
#Format columns
city_data_df['Lat']=city_data_df['Lat'].map("{:.2f}".format)
city_data_df['Lng']=city_data_df['Lng'].map('{:.2f}'.format)
city_data_df['Max Temp']=city_data_df['Max Temp'].map('{:.2f}'.format)
city_data_df['Rain (inches)']=city_data_df['Rain (inches)'].map('{:.1f}'.format)
city_data_df['Snow (inches)']=city_data_df['Snow (inches)'].map('{:.1f}'.format)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Hilo,US,19.73,-155.09,78.80,61,1,4.70,clear sky,0.0,0.0
1,Te Anau,NZ,-45.42,167.72,46.06,79,3,3.09,clear sky,0.0,0.0
2,Agadez,NE,19.75,10.25,88.11,16,5,7.25,clear sky,0.0,0.0
3,Byron Bay,AU,-28.65,153.62,75.20,53,29,10.29,scattered clouds,0.0,0.0
4,Hobart,AU,-42.88,147.33,59.00,54,75,5.82,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
561,Boke,GN,10.94,-14.30,72.82,95,100,2.30,moderate rain,1.1,0.0
562,Metro,ID,-5.11,105.31,88.16,64,89,4.45,light rain,0.0,0.0
563,Praia Da Vitoria,PT,38.73,-27.07,62.60,72,40,17.18,scattered clouds,0.0,0.0
564,Lazarev,RU,52.23,141.51,39.78,96,96,13.53,overcast clouds,0.0,0.0


In [85]:
city_data_df.dtypes

City                    object
Country                 object
Lat                     object
Lng                     object
Max Temp                object
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)           object
Snow (inches)           object
dtype: object

In [81]:
# Create the output file (CSV).
output_data_file='data/WeatherPy_challenge.csv'
# Export the City_data into a CSV
city_data_df.to_csv(output_data_file,index=False)

In [86]:
city_data_df['Rain (inches)']=city_data_df['Rain (inches)'].astype(str).astype(float)
city_data_df['Snow (inches)']=city_data_df['Snow (inches)'].astype(str).astype(float)

In [87]:
# Answer this question using Pandas methods: How many cities have recorded rainfall or snow?
print(f"There were {city_data_df[(city_data_df['Rain (inches)']>0) | (city_data_df['Snow (inches)']>0)].count()['City']} cities with rainfall or snow")


There were 31 cities with rainfall or snow
